In [ ]:
import os
from dotenv import load_dotenv
from langchain.groq import ChatGroq
import re
import httpx


In [ ]:
load_dotenv()

In [ ]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os. environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os. environ["LANGCHAIN_TRACING_V2"] = "true"
os. environ["LANGCHAIN_ENDPOINT"] = "https://api.smith. langchain.com"
os. environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [ ]:
groq_llm = ChatGroq(model_name="llama-groq-70b-8192-tool-use-preview")

groq_llm.invoke("Hi").content

In [ ]:
message = [{"role": "system", "content":"You are a helpful assistant"},
           {"role": "user", "content": "Hi, How are you?"}]

In [ ]:
result = groq_llm.invoke(message)

print(result.content)

In [ ]:
class ChatBot():
    
    def __init__(self, system=""):
        self.system = system
        self.message = []
        if self.system:
            self.message.append({"role": "system", "content":system})

    def __call__(self, message):
        self.message.append({"role": "user", "content":message})
        result = self.execute()
        self.message.append({"role": "assistant", "content": result})

        return result

    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)

        return result.content


In [ ]:
prompt = """

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: france is a country, the capital is Paris.

You then output:
Answer: The Capital of France is Paris

Please note: if you get basic conversation questions like 'hi', 'hello', 'how are you?',\n
you have to answer 'hi', 'hello', 'i am good'

""".strip()

In [ ]:
prompt_1 = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer related to fitness, health, or exercise.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 1800 * 0.2
Runs a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.

fitness_wiki:
e.g. fitness_wiki: Benefits of Cardio
Returns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).

exercise_plan:
e.g. exercise_plan: Beginner full-body workout
Returns a recommended workout plan based on the type of exercise or user level.

nutrition_info:
e.g. nutrition_info: Protein sources for vegetarians
Looks up nutritional information or suggestions for diets.

Example session:
Question: What are the benefits of cardio exercises?
Thought: I should look up information about the benefits of cardio exercises.
Action: fitness_wiki: Benefits of Cardio
PAUSE

You will be called again with this:

Observation: Cardio exercises improve cardiovascular health, increase lung capacity, and aid in weight loss.

You then output:

Answer: Cardio exercises improve cardiovascular health, increase lung capacity, and help with weight loss.
""".strip()

In [ ]:
action_re = re.compile("^Action: (\w+): (.*)")

In [6]:
def wikipedia(query):
    response = httpx.get("https://en/wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })

    return response.json()["query"]["search"][0]["snippet"]

In [ ]:
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """"
        select
        blog_entry.title || ': '|| substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
        blog_entry.created
        from
        blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
        blog_entry_fts match escape_fts(:q)
        order by
        blog_entry_fts.rank
        limit
        1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [1]:
def calculator(number):
    return eval(number)

In [4]:
calculator("3")

3

In [ ]:
known_actions = {
    "wikipedia": wikipedia,
    "calculator": calculator,
    "simon_blog_search": simon_blog_search
}

In [ ]:
def query(question, max_turen=5):
    i = 0
    bot = ChatBot(prompt)

    next_prompt = question

    while i < max_turen:
        i += 1
        result = bot(next_prompt)
        print(result)

        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]

        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            
            print("-- running {} {}".format(action, action_input))

            observation = known_actions[action](action_input)
            print("Observation: ", observation)
            next_prompt = f"Observation: {observation}"

        else:
            return result

In [ ]:
next_prompt = "Tell me about quantum computing from wikipedia"

In [ ]:
bot = ChatBot(prompt)

In [ ]:
bot(next_prompt)

In [ ]:
query("hi")

In [ ]:
query("How are you?")

In [ ]:
query("Fifteen * twenty five")

In [ ]:
query("Has Simon written about AI?")

In [ ]:
query("tell me the capital of USA")